# Turbine isentropic efficiency

A steam turbine performs with an isentropic efficiency of $\eta_t = 0.84$. The inlet conditions are 4 MPa and 650°C, with a mass flow rate of 100 kg/s, and the exit pressure is 10 kPa. Assume the turbine is adiabatic.

![Turbine](../../images/turbine.png)

**Problem:**
 - Determine the power produced by the turbine
 - Determine the rate of entropy generation

In [1]:
import numpy as np
import cantera as ct

from pint import UnitRegistry
ureg = UnitRegistry()
Q_ = ureg.Quantity

We can start by specifying state 1 and the other known quantities:

In [3]:
temp1 = Q_(650, 'degC')
pres1 = Q_(4, 'MPa')

state1 = ct.Water()
state1.TP = temp1.to('K').magnitude, pres1.to('Pa').magnitude

mass_flow_rate = Q_(100, 'kg/s')
efficiency = 0.84

pres2 = Q_(10, 'kPa')

To apply the isentropic efficiency, we'll need to separately consider the real turbine and an equivalent turbine operating in a reversible manner. They have the same initial conditions and mass flow rate.

For the reversible turbine, an entropy balance gives:
\begin{equation}
s_{s,2} = s_1
\end{equation}
and then with $P_2$ and $s_{s,2}$ we can fix state 2 for the reversible turbine:

In [6]:
state2_rev = ct.Water()
state2_rev.SP = state1.s, pres2.to('Pa').magnitude
state2_rev()


  water:

       temperature         319.003  K
          pressure           10000  Pa
           density         0.07464  kg/m^3
  mean mol. weight          18.016  amu
    vapor fraction        0.912916

                          1 kg            1 kmol
                       -----------      ------------
          enthalpy    -1.35945e+07       -2.449e+08     J
   internal energy    -1.37284e+07       -2.473e+08     J
           entropy         11017.2        1.985e+05     J/K
    Gibbs function     -1.7109e+07       -3.082e+08     J
 heat capacity c_p    inf              inf              J/K
 heat capacity c_v          100201        1.805e+06     J/K



Then, we can do an energy balance for the reversible turbine, which is also steady state and adiabatic:
\begin{equation}
\dot{m} h_1 = \dot{m} h_{s,2} + \dot{W}_{s,t}
\end{equation}
Then, recall that the isentropic efficiency is defined as
\begin{equation}
\eta_t = \frac{\dot{W}_t}{\dot{W}_{s,t}} \;,
\end{equation}
so we can obtain the actual turbine work using $\dot{W}_t = \eta_t \dot{W}_{s,t}$ :

In [8]:
work_isentropic = mass_flow_rate * (
    Q_(state1.h, 'J/kg') - Q_(state2_rev.h, 'J/kg')
    )
work_actual = efficiency * work_isentropic
print(f'Actual turbine work: {work_actual.to(ureg.megawatt): .2f}')

Actual turbine work: 118.73 megawatt


Then, we can perform an energy balance on the actual turbine:
\begin{equation}
\dot{m} h_1 = \dot{m} h_2 + \dot{W}_t \;,
\end{equation}
which we can use with the exit pressure to fix state 2.

In [9]:
enthalpy2 = Q_(state1.h, 'J/kg') - (work_actual / mass_flow_rate)

state2 = ct.Water()
state2.HP = enthalpy2.to('J/kg').magnitude, pres2.to('Pa').magnitude
state2()


  water:

       temperature         328.393  K
          pressure           10000  Pa
           density       0.0661646  kg/m^3
  mean mol. weight          18.016  amu
    vapor fraction               1

                          1 kg            1 kmol
                       -----------      ------------
          enthalpy    -1.33683e+07       -2.408e+08     J
   internal energy    -1.35194e+07       -2.436e+08     J
           entropy         11725.3        2.112e+05     J/K
    Gibbs function    -1.72188e+07       -3.102e+08     J
 heat capacity c_p         1894.36        3.413e+04     J/K
 heat capacity c_v         1423.02        2.564e+04     J/K



Finally, we can perform an entropy balance on the actual turbine:
\begin{equation}
\dot{m} s_1 + \dot{S}_{\text{gen}} = \dot{m} s_2 \;,
\end{equation}
which allows us to find the rate of entropy generation.

In [11]:
entropy_gen = mass_flow_rate * (
    Q_(state2.s, 'J/(kg K)') - Q_(state1.s, 'J/(kg K)')
    )
print(f'rate of entropy generation: {entropy_gen.to("kW/K"): .2f}')

rate of entropy generation: 70.81 kilowatt / kelvin
